In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/martinwg/ISA591/main/data/loan_default.csv', encoding = 'unicode_escape')  ## encode = "latin"

## **Predicting the Probability of Defaulting in a Loan**

In [ ]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,2,0.0,0.0,0.0,36 months,0.00,0.00,< 1 year,...,1.0,0.000000,0.00,0.00,0.00,Jan-07,0.00,Jan-07,Jan-07,1
1,3,545583,703644,2500.0,2500.0,2500.0,36 months,13.98,85.42,4 years,...,10.0,3075.291779,3075.29,2500.00,575.29,Jul-13,90.85,Aug-13,Jun-16,0
2,4,532101,687836,5000.0,5000.0,5000.0,36 months,15.95,175.67,4 years,...,15.0,2948.760000,2948.76,1909.02,873.81,Nov-11,175.67,NaN,Mar-12,1
3,5,877788,1092507,7000.0,7000.0,7000.0,36 months,9.91,225.58,10+ years,...,20.0,8082.391880,8082.39,7000.00,1082.39,Mar-14,1550.27,NaN,Mar-14,0
4,6,875406,1089981,2000.0,2000.0,2000.0,36 months,5.42,60.32,10+ years,...,15.0,2161.663244,2161.66,2000.00,161.66,Feb-14,53.12,NaN,Jun-16,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38480 entries, 0 to 38479
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              38480 non-null  int64  
 1   id                      38480 non-null  int64  
 2   member_id               38480 non-null  int64  
 3   loan_amnt               38479 non-null  float64
 4   funded_amnt             38479 non-null  float64
 5   funded_amnt_inv         38479 non-null  float64
 6   term                    38480 non-null  object 
 7   int_rate                38480 non-null  float64
 8   installment             38479 non-null  float64
 9   emp_length              37487 non-null  object 
 10  home_ownership          38480 non-null  object 
 11  annual_inc              38478 non-null  float64
 12  verification_status     38480 non-null  object 
 13  issue_d                 38480 non-null  object 
 14  loan_status             38480 non-null

## **Remove Unnecessary Variables**

* Uninformative (ID, name, zip code, state)
* Give information about the loan status


In [ ]:
## Variables by Type
## object variables (categoricals)
df.select_dtypes('O')

,term,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,earliest_cr_line,revol_util,last_pymnt_d,next_pymnt_d,last_credit_pull_d
0,36 months,< 1 year,RENT,Not Verified,Dec-99,Charged Off,major_purchase,000xx,ZZ,Jan-01,0.00%,Jan-07,Jan-07,Jan-07
1,36 months,4 years,RENT,Not Verified,Jul-10,Does not meet the credit policy. Status:Fully ...,other,487xx,MI,Aug-05,21.30%,Jul-13,Aug-13,Jun-16
2,36 months,4 years,RENT,Not Verified,Jun-10,Charged Off,debt_consolidation,115xx,NY,Apr-94,99.90%,Nov-11,NaN,Mar-12
3,36 months,10+ years,MORTGAGE,Not Verified,Sep-11,Fully Paid,other,751xx,TX,Mar-98,47.20%,Mar-14,NaN,Mar-14
4,36 months,10+ years,RENT,Not Verified,Sep-11,Fully Paid,debt_consolidation,112xx,NY,Jan-75,0%,Feb-14,NaN,Jun-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,36 months,3 years,RENT,Verified,Aug-11,Fully Paid,credit_card,805xx,CO,Jan-00,73.50%,Aug-14,NaN,Jun-16
38476,36 months,9 years,RENT,Source Verified,Aug-11,Fully Paid,credit_card,442xx,OH,Apr-99,39.70%,Sep-14,NaN,Jun-16
38477,60 months,10+ years,MORTGAGE,Source Verified,May-10,Fully Paid,debt_consolidation,073xx,NJ,Dec-95,62.20%,Jun-13,NaN,Feb-16
38478,36 months,1 year,RENT,Not Verified,Jul-11,Fully Paid,debt_consolidation,061xx,CT,Jul-07,53.30%,Aug-14,NaN,Sep-15


In [ ]:
## gives the response
##
pd.crosstab(df.loan_status, df.repay_fail)

repay_fail,0,1
loan_status,,
Charged Off,0,5111
Current,867,0
Default,0,2
Does not meet the credit policy. Status:Charged Off,0,689
Does not meet the credit policy. Status:Fully Paid,1782,0
Fully Paid,29987,0
In Grace Period,15,0
Late (16-30 days),0,7
Late (31-120 days),0,20


## **Drop the following variables**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38480 entries, 0 to 38479
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              38480 non-null  int64  
 1   id                      38480 non-null  int64  
 2   member_id               38480 non-null  int64  
 3   loan_amnt               38479 non-null  float64
 4   funded_amnt             38479 non-null  float64
 5   funded_amnt_inv         38479 non-null  float64
 6   term                    38480 non-null  object 
 7   int_rate                38480 non-null  float64
 8   installment             38479 non-null  float64
 9   emp_length              37487 non-null  object 
 10  home_ownership          38480 non-null  object 
 11  annual_inc              38478 non-null  float64
 12  verification_status     38480 non-null  object 
 13  issue_d                 38480 non-null  object 
 14  loan_status             38480 non-null

In [ ]:
df.drop(['Unnamed: 0', 'id', 'member_id', 'zip_code', 'addr_state', 'last_pymnt_d', 'next_pymnt_d', 'total_rec_prncp'], axis = 1, inplace = True)

In [ ]:
df.drop('loan_status', axis = 1, inplace = True)

## **Check on Dimension Reduction**

* check the levels of categoricals variables

In [ ]:
df.select_dtypes('O')

,term,emp_length,home_ownership,verification_status,issue_d,purpose,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,< 1 year,RENT,Not Verified,Dec-99,major_purchase,Jan-01,0.00%,Jan-07
1,36 months,4 years,RENT,Not Verified,Jul-10,other,Aug-05,21.30%,Jun-16
2,36 months,4 years,RENT,Not Verified,Jun-10,debt_consolidation,Apr-94,99.90%,Mar-12
3,36 months,10+ years,MORTGAGE,Not Verified,Sep-11,other,Mar-98,47.20%,Mar-14
4,36 months,10+ years,RENT,Not Verified,Sep-11,debt_consolidation,Jan-75,0%,Jun-16
...,...,...,...,...,...,...,...,...,...
38475,36 months,3 years,RENT,Verified,Aug-11,credit_card,Jan-00,73.50%,Jun-16
38476,36 months,9 years,RENT,Source Verified,Aug-11,credit_card,Apr-99,39.70%,Jun-16
38477,60 months,10+ years,MORTGAGE,Source Verified,May-10,debt_consolidation,Dec-95,62.20%,Feb-16
38478,36 months,1 year,RENT,Not Verified,Jul-11,debt_consolidation,Jul-07,53.30%,Sep-15


In [ ]:
## Check the levels of ALL variables
for i in df.select_dtypes('O').columns:
  print(i, df.select_dtypes('O')[i].nunique())

term 2
emp_length 11
home_ownership 5
verification_status 3
issue_d 57
purpose 14
earliest_cr_line 528
revol_util 1109
last_credit_pull_d 110


## **Work with Dates**

In [ ]:
df['issue_d'] = pd.to_datetime(df.issue_d, format = "%b-%y")
df['earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, format = "%b-%y")
df['last_credit_pull_d'] = pd.to_datetime(df.last_credit_pull_d, format = "%b-%y")

## **Change the Dates to Number of Days From**

In [ ]:
df['issue_d'] = (pd.Timestamp.today() - df['issue_d']).dt.days
df['earliest_cr_line'] = (pd.Timestamp.today() - df['earliest_cr_line']).dt.days
df['last_credit_pull_d'] = (pd.Timestamp.today() - df['last_credit_pull_d']).dt.days

## **Fix % or $**

In [ ]:
df['revol_util'] = df.revol_util.str.extract("(\d+)", expand = False)

## **Missing Values**

* Imputation
* get rid of some

In [ ]:
df.isna().sum()

loan_amnt                     1
funded_amnt                   1
funded_amnt_inv               1
term                          0
int_rate                      0
installment                   1
emp_length                  993
home_ownership                0
annual_inc                    2
verification_status           0
issue_d                       0
purpose                       0
dti                           0
delinq_2yrs                   1
earliest_cr_line              0
inq_last_6mths                1
mths_since_last_delinq    24363
open_acc                      1
pub_rec                       1
revol_bal                     4
revol_util                   59
total_acc                     1
total_pymnt                   1
total_pymnt_inv               1
total_rec_int                 1
last_pymnt_amnt               1
last_credit_pull_d            3
repay_fail                    0
dtype: int64

In [ ]:
df.mths_since_last_delinq

0         NaN
1         NaN
2        59.0
3         3.0
4        72.0
         ... 
38475     NaN
38476     NaN
38477     NaN
38478     NaN
38479    38.0
Name: mths_since_last_delinq, Length: 38480, dtype: float64

## **Binning for Delinquencies**

In [ ]:
df.loc[df['mths_since_last_delinq'].between(0,12, "both"), 'LAST_DELINQ'] = '0-12 months'
df.loc[df['mths_since_last_delinq'].between(12,24, "right"), 'LAST_DELINQ'] = '12-24 months'
df.loc[df['mths_since_last_delinq'].between(24,36, "right"), 'LAST_DELINQ'] = '24-36 months'
df.loc[df['mths_since_last_delinq'].between(36,48, "right"), 'LAST_DELINQ'] = '36-48 months'
df.loc[df['mths_since_last_delinq'].between(48,df['mths_since_last_delinq'].max(), "right"), 'LAST_DELINQ'] = '48-LONG months'

In [ ]:
df.LAST_DELINQ.fillna("Never Defaulted", inplace = True)

In [ ]:
df.drop('mths_since_last_delinq', axis = 1, inplace = True)

In [ ]:
df.isna().sum()

loan_amnt                1
funded_amnt              1
funded_amnt_inv          1
term                     0
int_rate                 0
installment              1
emp_length             993
home_ownership           0
annual_inc               2
verification_status      0
issue_d                  0
purpose                  0
dti                      0
delinq_2yrs              1
earliest_cr_line         0
inq_last_6mths           1
open_acc                 1
pub_rec                  1
revol_bal                4
revol_util              59
total_acc                1
total_pymnt              1
total_pymnt_inv          1
total_rec_int            1
last_pymnt_amnt          1
last_credit_pull_d       3
repay_fail               0
LAST_DELINQ              0
dtype: int64

In [ ]:
df.emp_length.fillna("unknown", inplace = True)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,...,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt,last_credit_pull_d,repay_fail,LAST_DELINQ
0,0.0,0.0,0.0,36 months,0.00,0.00,< 1 year,RENT,0.0,Not Verified,...,0.0,0,1.0,0.000000,0.00,0.00,0.00,6148.0,1,Never Defaulted
1,2500.0,2500.0,2500.0,36 months,13.98,85.42,4 years,RENT,20004.0,Not Verified,...,981.0,21,10.0,3075.291779,3075.29,575.29,90.85,2709.0,0,Never Defaulted
2,5000.0,5000.0,5000.0,36 months,15.95,175.67,4 years,RENT,59000.0,Not Verified,...,18773.0,99,15.0,2948.760000,2948.76,873.81,175.67,4262.0,1,48-LONG months
3,7000.0,7000.0,7000.0,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,...,3269.0,47,20.0,8082.391880,8082.39,1082.39,1550.27,3532.0,0,0-12 months
4,2000.0,2000.0,2000.0,36 months,5.42,60.32,10+ years,RENT,30000.0,Not Verified,...,0.0,0,15.0,2161.663244,2161.66,161.66,53.12,2709.0,0,48-LONG months


In [ ]:
df['revol_util'] = df.revol_util.astype('int')

## **Dummy-Encoding**

In [ ]:
df = pd.get_dummies(df, drop_first = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38413 entries, 0 to 38479
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            38413 non-null  float64
 1   funded_amnt                          38413 non-null  float64
 2   funded_amnt_inv                      38413 non-null  float64
 3   int_rate                             38413 non-null  float64
 4   installment                          38413 non-null  float64
 5   annual_inc                           38413 non-null  float64
 6   issue_d                              38413 non-null  int64  
 7   dti                                  38413 non-null  float64
 8   delinq_2yrs                          38413 non-null  float64
 9   earliest_cr_line                     38413 non-null  int64  
 10  inq_last_6mths                       38413 non-null  float64
 11  open_acc                    

## **TO DO: Check on GridSearch CV**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
grid = {'max_depth': [2,4,6,10, None],
        'max_leaf_nodes': [10, 20, 30, 60, None],
        'criterion': ['gini', 'entropy']}

In [ ]:
grid

{'max_depth': [2, 4, 6, 10, None],
 'max_leaf_nodes': [10, 20, 30, 60, None],
 'criterion': ['gini', 'entropy']}

In [ ]:
dt = GridSearchCV(DecisionTreeClassifier(), param_grid = grid, cv = 5, n_jobs = -1)

In [ ]:
df.to_csv('loan_default_clean.csv', index = False)